# 🎵 Bots-n-Cats Monitoring Dashboard

Real-time monitoring for your GitHub → Music system

In [1]:
# Setup and imports
import asyncio
import subprocess
import json
import requests
from datetime import datetime
from IPython.display import display, HTML, clear_output
import time

## 📊 System Status Overview

In [2]:
def get_system_status():
    """Get current status of all services"""
    status = {}
    
    # Check webhook server
    try:
        r = requests.get('http://localhost:3000/health', timeout=2)
        status['webhook_server'] = '🟢 Running' if r.status_code == 200 else '🔴 Error'
    except:
        status['webhook_server'] = '🔴 Offline'
    
    # Check ngrok
    try:
        r = requests.get('http://localhost:4040/api/tunnels', timeout=2)
        data = r.json()
        if data.get('tunnels'):
            tunnel = data['tunnels'][0]
            status['ngrok'] = f"🟢 Running"
            status['ngrok_url'] = tunnel['public_url']
        else:
            status['ngrok'] = '🔴 No tunnels'
    except:
        status['ngrok'] = '🔴 Offline'
    
    # Check streaming server
    try:
        r = requests.get('http://localhost:3001/health', timeout=2)
        status['streaming_server'] = '🟢 Running' if r.status_code == 200 else '🔴 Error'
    except:
        status['streaming_server'] = '🔴 Offline'
    
    return status

# Display status
status = get_system_status()
display(HTML(f"""
<div style="font-family: monospace; background: #1e1e1e; color: #d4d4d4; padding: 20px; border-radius: 8px;">
    <h3 style="color: #4ec9b0;">System Status - {datetime.now().strftime('%H:%M:%S')}</h3>
    <table style="width: 100%; color: #d4d4d4;">
        <tr><td><b>Webhook Server:</b></td><td>{status.get('webhook_server', '❓ Unknown')}</td></tr>
        <tr><td><b>Ngrok Tunnel:</b></td><td>{status.get('ngrok', '❓ Unknown')}</td></tr>
        <tr><td><b>Public URL:</b></td><td>{status.get('ngrok_url', 'N/A')}</td></tr>
        <tr><td><b>Streaming Server:</b></td><td>{status.get('streaming_server', '❓ Unknown')}</td></tr>
    </table>
</div>
"""))

Webhook Server:,🟢 Running
Ngrok Tunnel:,🟢 Running
Public URL:,https://ed7a2dd6a687.ngrok.app
Streaming Server:,🟢 Running


## 📡 Live Webhook Server Logs

**Run this cell and leave it running to see webhook events in real-time!**

In [6]:
async def tail_webhook_logs(duration=60):
    """Tail webhook server logs for specified duration (seconds)"""
    print(f"📡 Monitoring webhook-server.log for {duration} seconds...")
    print(f"{'='*80}\n")
    
    # Start from end of file
    proc = await asyncio.create_subprocess_exec(
        'tail', '-f', '/tmp/webhook-server.log',
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )
    
    start_time = asyncio.get_event_loop().time()
    
    try:
        while asyncio.get_event_loop().time() - start_time < duration:
            try:
                line = await asyncio.wait_for(proc.stdout.readline(), timeout=1.0)
                if line:
                    decoded = line.decode().strip()
                    # Color code different log types
                    if 'ERROR' in decoded:
                        print(f"\033[91m{decoded}\033[0m")  # Red
                    elif 'INFO' in decoded and 'Webhook event received' in decoded:
                        print(f"\033[92m🎵 {decoded}\033[0m")  # Green
                    elif 'WARN' in decoded:
                        print(f"\033[93m{decoded}\033[0m")  # Yellow
                    else:
                        print(decoded)
            except asyncio.TimeoutError:
                continue
    finally:
        proc.terminate()
        await proc.wait()
    
    print(f"\n{'='*80}")
    print(f"Monitoring stopped after {duration} seconds")

# Run for 60 seconds (adjust as needed)
await tail_webhook_logs(duration=60)

📡 Monitoring webhook-server.log for 60 seconds...

🎵 [2025-10-18T08:21:50.465Z] INFO: Webhook event received {
"eventType": "ping",
"action": "unknown",
"deliveryId": "7e281532-abfb-11f0-9e32-5032a6a58bc9",
"repository": "I-m-A-g-I-n-E/bots-n-cats",
"sender": "research-developer"
}
[2025-10-18T08:21:50.465Z] INFO: Unsupported event type {
"error": "Unsupported event type: ping"
}

Monitoring stopped after 60 seconds


## 🌐 Recent GitHub Webhook Deliveries

In [7]:
def get_recent_deliveries():
    """Get recent webhook deliveries from GitHub"""
    try:
        result = subprocess.run(
            ['gh', 'api', 'repos/I-m-A-g-I-n-E/bots-n-cats/hooks/575942860/deliveries', '--paginate'],
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            deliveries = json.loads(result.stdout)
            
            html = '<div style="font-family: monospace; background: #1e1e1e; color: #d4d4d4; padding: 20px; border-radius: 8px;">'
            html += f'<h3 style="color: #4ec9b0;">Recent Webhook Deliveries ({len(deliveries[:10])})</h3>'
            html += '<table style="width: 100%; color: #d4d4d4; border-collapse: collapse;">'
            html += '<tr style="border-bottom: 1px solid #444;"><th>Time</th><th>Event</th><th>Status</th><th>Duration</th></tr>'
            
            for delivery in deliveries[:10]:
                delivered_at = delivery.get('delivered_at', 'N/A')
                event = delivery.get('event', 'unknown')
                status = delivery.get('status', 'pending')
                duration = delivery.get('duration', 0)
                
                status_icon = '🟢' if status == 'OK' else '🔴' if status == 'error' else '🟡'
                
                html += f'<tr style="border-bottom: 1px solid #333;">'
                html += f'<td>{delivered_at[:19] if delivered_at != "N/A" else "N/A"}</td>'
                html += f'<td><b>{event}</b></td>'
                html += f'<td>{status_icon} {status}</td>'
                html += f'<td>{duration:.2f}s</td>'
                html += '</tr>'
            
            html += '</table></div>'
            display(HTML(html))
        else:
            print(f"Error fetching deliveries: {result.stderr}")
    except Exception as e:
        print(f"Error: {e}")

get_recent_deliveries()

Error: Expecting value: line 1 column 1 (char 0)


## 🔄 Auto-Refreshing Dashboard

**This cell will refresh every 5 seconds. Stop it with the ⏹ button.**

In [4]:
async def live_dashboard(refresh_interval=5, duration=300):
    """Live dashboard that refreshes every N seconds"""
    start_time = time.time()
    
    while time.time() - start_time < duration:
        clear_output(wait=True)
        
        # Get status
        status = get_system_status()
        
        # Get recent webhook events from log
        try:
            result = subprocess.run(
                ['tail', '-20', '/tmp/webhook-server.log'],
                capture_output=True,
                text=True
            )
            recent_logs = result.stdout.split('\n')[-10:]
        except:
            recent_logs = ['No logs available']
        
        # Build HTML dashboard
        html = f"""
        <div style="font-family: monospace; background: #1e1e1e; color: #d4d4d4; padding: 20px; border-radius: 8px;">
            <h2 style="color: #4ec9b0;">🎵 Bots-n-Cats Live Dashboard</h2>
            <p style="color: #888;">Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
            
            <h3 style="color: #569cd6;">System Status</h3>
            <table style="width: 100%; color: #d4d4d4;">
                <tr><td><b>Webhook Server:</b></td><td>{status.get('webhook_server', '❓ Unknown')}</td></tr>
                <tr><td><b>Ngrok Tunnel:</b></td><td>{status.get('ngrok', '❓ Unknown')}</td></tr>
                <tr><td><b>Public URL:</b></td><td style="color: #4ec9b0;">{status.get('ngrok_url', 'N/A')}</td></tr>
                <tr><td><b>Streaming Server:</b></td><td>{status.get('streaming_server', '❓ Unknown')}</td></tr>
            </table>
            
            <h3 style="color: #569cd6; margin-top: 20px;">Recent Activity</h3>
            <div style="background: #252526; padding: 10px; border-radius: 4px; font-size: 12px; max-height: 300px; overflow-y: auto;">
        """
        
        for log in recent_logs:
            if log.strip():
                if 'ERROR' in log:
                    html += f'<div style="color: #f48771;">🔴 {log}</div>'
                elif 'Webhook event received' in log:
                    html += f'<div style="color: #4ec9b0;">🎵 {log}</div>'
                elif 'INFO' in log:
                    html += f'<div style="color: #d4d4d4;">{log}</div>'
                else:
                    html += f'<div style="color: #888;">{log}</div>'
        
        html += """
            </div>
        </div>
        """
        
        display(HTML(html))
        
        await asyncio.sleep(refresh_interval)
    
    print("\nDashboard stopped")

# Run for 5 minutes (300 seconds)
await live_dashboard(refresh_interval=5, duration=300)

Webhook Server:,🟢 Running
Ngrok Tunnel:,🟢 Running
Public URL:,https://ed7a2dd6a687.ngrok.app
Streaming Server:,🟢 Running


CancelledError: 

## 🧪 Test Webhook

Send a test ping to your webhook:

In [ ]:
def send_test_ping():
    """Send a test ping to the webhook"""
    result = subprocess.run(
        ['gh', 'api', 'repos/I-m-A-g-I-n-E/bots-n-cats/hooks/575942860/pings', '-X', 'POST'],
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0:
        print("✅ Test ping sent successfully!")
        print("Check the 'Live Webhook Server Logs' cell above to see it arrive.")
    else:
        print(f"❌ Error: {result.stderr}")

send_test_ping()

## 📊 Webhook Statistics

In [ ]:
def get_webhook_stats():
    """Get statistics from webhook deliveries"""
    try:
        result = subprocess.run(
            ['gh', 'api', 'repos/I-m-A-g-I-n-E/bots-n-cats/hooks/575942860/deliveries'],
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            deliveries = json.loads(result.stdout)
            
            # Count by event type
            event_counts = {}
            for d in deliveries:
                event = d.get('event', 'unknown')
                event_counts[event] = event_counts.get(event, 0) + 1
            
            # Calculate success rate
            total = len(deliveries)
            successful = sum(1 for d in deliveries if d.get('status') == 'OK')
            success_rate = (successful / total * 100) if total > 0 else 0
            
            html = '<div style="font-family: monospace; background: #1e1e1e; color: #d4d4d4; padding: 20px; border-radius: 8px;">'
            html += '<h3 style="color: #4ec9b0;">Webhook Statistics</h3>'
            html += f'<p><b>Total Deliveries:</b> {total}</p>'
            html += f'<p><b>Success Rate:</b> {success_rate:.1f}% ({successful}/{total})</p>'
            html += '<h4 style="color: #569cd6;">Events by Type:</h4>'
            html += '<ul>'
            for event, count in sorted(event_counts.items(), key=lambda x: x[1], reverse=True):
                html += f'<li><b>{event}:</b> {count}</li>'
            html += '</ul></div>'
            
            display(HTML(html))
        else:
            print(f"Error: {result.stderr}")
    except Exception as e:
        print(f"Error: {e}")

get_webhook_stats()

## 🛠️ Quick Commands

In [ ]:
# View last 50 lines of webhook logs
!tail -50 /tmp/webhook-server.log

In [ ]:
# Check ngrok tunnels
!curl -s http://localhost:4040/api/tunnels | python3 -m json.tool

In [ ]:
# Check webhook server health
!curl -s http://localhost:3000/health | python3 -m json.tool